<a href="https://colab.research.google.com/github/aadityaverma/3DDFA/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



In [1]:
!nvidia-smi

Sun Nov 13 07:39:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    41W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
%%capture
!pip install -U asteroid
!pip install -U pydub

## Pretrained model speech separation

Asteroid comes with [pretrained models](https://huggingface.co/models?filter=asteroid) that help you get started quickly, like the 8 kHz [DPRNN](https://huggingface.co/mpariente/DPRNNTasNet-ks2_WHAM_sepclean).

As an example audio file to separate, you can use a mixture from the Deep clustering paper. Let's download it and listen to the unseparated version:

In [ ]:
from IPython.display import display, Audio

display(Audio("CF2101835629.wav"))
#/content/CF2101835629.wav

#%%
from asteroid.models import BaseModel
import soundfile as sf

# 'from_pretrained' automatically uses the right model class (asteroid.models.DPRNNTasNet).
model = BaseModel.from_pretrained("mpariente/DPRNNTasNet-ks2_WHAM_sepclean")
#%%
#Importing library and thir function
from pydub import AudioSegment
from pydub.silence import split_on_silence

#reading from audio mp3 file
sound = AudioSegment.from_wav("/content/CF2101835629.wav")



In [ ]:
#@title

# spliting audio files
audio_chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-40 )

#loop is used to iterate over the output list
for i, chunk in enumerate(audio_chunks):
    output_file = "/Users/aditya/Downloads/output/chunk{0}.wav".format(i)
    print("Exporting file", output_file)
    chunk.export(output_file, format="wav")

# chunk files saved as Output

In [ ]:
#@title
#%%

#loop is used to iterate over the output list
for i, chunk in enumerate(audio_chunks):
    output_file = "/Users/aditya/Downloads/output/chunk{0}.wav".format(i)
    print("Seperating Speaker ", output_file)
    model.separate(output_file)
#    model.separate("/Users/aditya/Downloads/output/chunk{0}.wav".format(i))

#    !asteroid-infer "mpariente/DPRNNTasNet-ks2_WHAM_sepclean" --files "/Users/aditya/Downloads/output/chunk{0}.wav".format(i)

#    chunk.export(output_file, format="mp3")


In [2]:
#%%
from asteroid.models import BaseModel
import soundfile as sf

# 'from_pretrained' automatically uses the right model class (asteroid.models.DPRNNTasNet).
model = BaseModel.from_pretrained("mpariente/DPRNNTasNet-ks2_WHAM_sepclean")


/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:591: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/14.7M [00:00<?, ?B/s]

In [3]:
#%%
from pydub import AudioSegment
import math

class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename

        self.audio = AudioSegment.from_wav(self.filepath)

    def get_duration(self):
        return self.audio.duration_seconds

    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '/out/' + split_filename, format="wav")

    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_mins - min_per_split:
                print('All splited successfully')




In [4]:
!mkdir '/content/out/'

In [5]:
#%%
folder = '/content/'
file = 'CF2101835629.wav'
split_wav = SplitWavAudioMubin(folder, file)
split_wav.multiple_split(min_per_split=1)


0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
All splited successfully


In [6]:
import glob
i = 0
for filepath in glob.iglob('/content/out/*.wav'):
    i = i + 1
    print(filepath)
    model.separate(filepath)
    

#loop is used to iterate over the output list
#for i  in enumerate(audio_chunks):
#    output_file = "/content/{0}_CF2101835629.wav".format(i)
#    print("Seperating Speaker ", output_file)
#    model.separate(output_file)
#    model.separate("/Users/aditya/Downloads/output/chunk{0}.wav".format(i))

#    !asteroid-infer "mpariente/DPRNNTasNet-ks2_WHAM_sepclean" --files "/Users/aditya/Downloads/output/chunk{0}.wav".format(i)

#    chunk.export(output_file, format="mp3")


/content/out/4_CF2101835629.wav
/content/out/6_CF2101835629.wav
/content/out/8_CF2101835629.wav
/content/out/11_CF2101835629.wav
/content/out/1_CF2101835629.wav
/content/out/2_CF2101835629.wav
/content/out/9_CF2101835629.wav
/content/out/3_CF2101835629.wav
/content/out/7_CF2101835629.wav
/content/out/10_CF2101835629.wav
/content/out/5_CF2101835629.wav
/content/out/0_CF2101835629.wav


In [15]:

import glob
from pydub import AudioSegment
from pydub.playback import play
i = 0

output_final1 = AudioSegment.from_file("/content/CF2101835629.wav")
output_final2 = AudioSegment.from_file("/content/CF2101835629.wav")
for filepath in glob.iglob('/content/out/*_est1.wav'):
    i = i + 1
    output_file1 = AudioSegment.from_file(filepath)
#    output_file2 = AudioSegment.from_file("filepath")
    print("Merging Speaker 1 est", output_file1)
    output_final1 = output_final1 + output_file1
    print("Exporting Speaker 1 est", output_file1)
    output_final1.export(out_f = "/content/Client.wav",
                         format = "wav")
#    print("Merging Speaker 2 est", output_file2)
#    output_final2 = output_final2 + output_file2
#    print("Exporting Speaker 2 est", output_file2)
#    output_final2.export(out_f = "/content/Agent.wav",
#                         format = "wav")
for filepath in glob.iglob('/content/out/*_est2.wav'):
    i = i + 1
#    output_file1 = AudioSegment.from_file(filepath)
    output_file2 = AudioSegment.from_file(filepath)
#    print("Merging Speaker 1 est", output_file1)
#    output_final1 = output_final1 + output_file1
#    print("Exporting Speaker 1 est", output_file1)
#    output_final1.export(out_f = "/content/Client.wav",
#                        format = "wav")
    print("Merging Speaker 2 est", output_file2)
    output_final2 = output_final2 + output_file2
    print("Exporting Speaker 2 est", output_file2)
    output_final2.export(out_f = "/content/Agent.wav",
                         format = "wav")    

#play(output_final1)
#    model.separate(output_file)

Merging Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cad0>
Exporting Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cad0>
Merging Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cf10>
Exporting Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cf10>
Merging Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cad0>
Exporting Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cad0>
Merging Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cf10>
Exporting Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3cf10>
Merging Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db60b2c10>
Exporting Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db60b2c10>
Merging Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3c650>
Exporting Speaker 1 est <pydub.audio_segment.AudioSegment object at 0x7f2db4a3c650>
Merg

In [2]:
from IPython.display import display, Audio

display(Audio("/content/CF2101835629.wav"))
#/content/CF2101835629.wav

In [1]:
from IPython.display import display, Audio

display(Audio("Agent.wav"))
#/content/CF2101835629.wav

In [17]:
from IPython.display import display, Audio

display(Audio("Client.wav"))
#/content/CF2101835629.wav

## Faster GPUs

Users who have purchased one of Colab's paid plans have access to premium GPUs. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to enable Premium accelerator. Subject to availability, selecting a premium GPU may grant you access to a V100 or A100 Nvidia GPU.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator dropdown to GPU.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape dropdown. After, re-execute the code cell.


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab up to 24 hours. To enable background execution on your notebook, in the menu navigate to `Runtime > Change runtime type` and enable Background execution.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb) 
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colaboratory makes possible, check out these  tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.
